In [6]:
!pip3 install pandas

    100% |################################| 9.5MB 185kB/s ta 0:00:011
    100% |################################| 512kB 2.5MB/s ta 0:00:011
    100% |################################| 13.4MB 133kB/s ta 0:00:011  2% |                                | 348kB 6.4MB/s eta 0:00:03    6% |##                              | 901kB 5.4MB/s eta 0:00:03    26% |########                        | 3.6MB 6.0MB/s eta 0:00:02    62% |####################            | 8.4MB 5.8MB/s eta 0:00:01


In [8]:
!pip3 install scipy 

    100% |################################| 25.9MB 66kB/s eta 0:00:011    76% |########################        | 19.8MB 4.0MB/s eta 0:00:02    85% |###########################     | 22.1MB 13.7MB/s eta 0:00:01    88% |############################    | 22.8MB 17.6MB/s eta 0:00:01


In [10]:
!pip3 install dwave-neal

    100% |################################| 399kB 2.2MB/s ta 0:00:01
    100% |################################| 6.1MB 277kB/s eta 0:00:01    59% |###################             | 3.6MB 8.5MB/s eta 0:00:01    80% |#########################       | 4.9MB 3.5MB/s eta 0:00:01


In [12]:
!pip3 install pyqubo

    100% |################################| 256kB 2.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for wrapt ... done
  Stored in directory: /root/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
Successfully built wrapt


In [13]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [14]:
ost = pd.read_csv("../input/ost.csv", sep=',', index_col=0)
ost
#論文と照らし合わせ済み

,LI,SEX,AOP,COUNT
Y,,,,
0,0,0,0,0
0,0,0,1,0
0,0,1,0,0
0,0,1,1,0
0,1,0,0,0
0,1,0,1,2
0,1,1,0,4
0,1,1,1,11
1,0,0,0,3


超幾何分布を考えると、$K=29, n=36, k=19, N=46$

マルコフ基底において固定する十分統計量は$29, 17, 46, 10, 36$（周辺和）

### マルコフ基底
- Risa/Asir:日本人が作った
- [Sympy](https://www.sympy.org/en/index.html)
  - [github](https://github.com/sympy/sympy)
  - [pypi](https://pypi.org/project/sympy/)
  - [グレブナー基底](https://docs.sympy.org/latest/modules/polys/index.html)

In [15]:
!pip3 install sympy

In [16]:
from __future__ import division

#from sympy import symbols, groebner, Function, expand
from sympy import *

In [17]:
x, y, z, t = symbols('x y z t')
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)

In [18]:
groebner([x**2 + 1, y**4*x + x**3], x, y, order='lex')#公式とoutputが異なる

GroebnerBasis([x**2 + 1, y**4 - 1], x, y, domain='ZZ', order='lex')

In [19]:
groebner([x**2 + 1, y**4*x + x**3, x*y*z**3], x, y, z, order='grevlex')#公式とoutputが異なる

GroebnerBasis([y**4 - 1, z**3, x**2 + 1], x, y, z, domain='ZZ', order='grevlex')

In [20]:
init_printing(use_unicode=True, wrap_line=False)

In [21]:
var('x,y,z,s,c,n')

(x, y, z, s, c, n)

In [22]:
f = expand((1 - c**2)**5 * (1 - s**2)**5 * (c**2 + s**2)**10)

In [23]:
groebner([f, c**2 + s**2 - 1])

             ⎛⎡ 2    2       20      18       16       14      12    10⎤                           ⎞
GroebnerBasis⎝⎣c  + s  - 1, c   - 5⋅c   + 10⋅c   - 10⋅c   + 5⋅c   - c  ⎦, s, c, domain=ℤ, order=lex⎠